# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis notebooks, we visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this notebook, we will be performing more interactive visual analytics using `Folium`.


## Objectives


We will perform the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completing the above tasks, we should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
#!pip3 install folium
#!pip3 install wget

In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1:


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [3]:
spacex_df = pd.read_csv('spacex_launch_geo.csv')

Now, we can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give us any intuitive insights about where those launch sites are. So let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center, we can just google to find out the coordinates!
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example:


In [6]:
# Let's create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Let's also create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Createing an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [7]:
# Initializing the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, we add a Circle object based on its coordinate (Lat, Long) values. In addition, we add Launch site name as a popup label
for i in [0,1,2,3]:
    Circle = folium.Circle((launch_sites_df['Lat'][i],launch_sites_df['Long'][i]),radius=50,color='#d35400',fill=True).add_child(folium.Popup(launch_sites_df['Launch Site'][i]))
    Marker = folium.Marker((launch_sites_df['Lat'][i],launch_sites_df['Long'][i]), icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df['Launch Site'][i], ))
    site_map.add_child(Circle)
    site_map.add_child(Marker)

site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2:


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [9]:
marker_cluster = MarkerCluster()


In [10]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [11]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
for i in range(0,len(spacex_df)):
    marker = folium.Marker([spacex_df['Lat'][i],spacex_df['Long'][i]],icon=folium.Icon(color='white', icon_color=spacex_df['marker_color'][i]))
    marker_cluster.add_child(marker)

site_map

Now we can interact with the output. From the color-labeled markers in marker clusters, we are able to easily identify which launch sites have relatively high success rates.


# TASK 3:


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now we can zoom in to a launch site and explore its proximity to see if we can easily find any railway, highway, coastline, etc. We can move the mouse to these points and mark down their coordinates. This will allow us to calculate their distance from the launch site.


We can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Let's mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [14]:
# find coordinate of the closet coastline
coastline_lat = 28.56231
coastline_lon = -80.56781 

#calculating the distance of the coastline from CCAFS LC-40 launch site:
launch_site_lat = launch_sites_df[launch_sites_df['Launch Site']=='CCAFS LC-40']['Lat'][0]
launch_site_lon = launch_sites_df[launch_sites_df['Launch Site']=='CCAFS LC-40']['Long'][0]

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

After obtaining its coordinate, we can create a `folium.Marker` to show the distance


In [15]:

distance_marker = folium.Marker(
    [coastline_lat,coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

site_map.add_child(distance_marker)

Now we can draw a `PolyLine` between a launch site to the selected coastline point


In [16]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(locations=[[launch_site_lat,launch_site_lon],[coastline_lat,coastline_lon]], weight=1)
site_map.add_child(lines)

Similarly, we can draw a line betwee a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find their coordinates on the map first.


In [17]:
# Creating a marker with distance to closest city
# Drawing a line between the marker to the launch site
city_lat= 28.10894
city_lon= -80.63731

distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

distance_marker = folium.Marker(
    [city_lat,city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(distance_marker)

lines = folium.PolyLine(locations=[[launch_site_lat,launch_site_lon],[city_lat,city_lon]], weight=1)
site_map.add_child(lines)

In [20]:
# Creating a marker with distance to closest railway
# Drawing a line between the marker to the launch site
railway_lat= 28.57227
railway_lon= -80.58526

distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

distance_marker = folium.Marker(
    [railway_lat,railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )
site_map.add_child(distance_marker)

lines = folium.PolyLine(locations=[[launch_site_lat,launch_site_lon],[railway_lat,railway_lon]], weight=1)
site_map.add_child(lines)

In [18]:
# Creating a marker with distance to closest highway
# Drawing a line between the marker to the launch site
highway_lat= 28.56321
highway_lon= -80.57075

distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

distance_marker = folium.Marker(
    [highway_lat,highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
    )
site_map.add_child(distance_marker)

lines = folium.PolyLine(locations=[[launch_site_lat,launch_site_lon],[highway_lat,highway_lon]], weight=1)
site_map.add_child(lines)

Insights: Launch sites seem to keep a cetarin distance away from the cities for safety. However, they are close to railways, highways, and coastlies!

Through the interactive visualizations, we discovered many interesting insights related to the launch sites' location using folium!
